## 환경설정

1. http://braincrew2.iptime.org:8001 에 접속하여 회원가입해 주세요. (비밀번호는 단순하게 기입하시는 것을 권장 드려요. 예. 1234)
2. `username` 에 이메일 형식의 아이디를 기입해 주세요.
3. `password` 에 비밀번호를 기입해 주세요.


In [ ]:
project = "CATSvsDOGS"  # 수정하지 마세요
username = ""  # 이메일아이디 (예시. abc@hello.com)
password = ""  # 비밀번호

아래의 코드를 순서대로 실행해 주세요.


In [ ]:
import os
import requests

if not os.path.exists("competition.py"):
    url = "https://link.teddynote.com/COMPT"
    file_name = "competition.py"
    response = requests.get(url)
    with open(file_name, "wb") as file:
        file.write(response.content)

아래 코드를 실행하여 데이터를 다운로드 받습니다.


In [ ]:
import competition

# 파일 다운로드
competition.download_competition_files(
    f"https://link.teddynote.com/{project}", use_competition_url=False
)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os

# Data 경로 설정
DATA_DIR = "data"

# 경고 무시
warnings.filterwarnings("ignore")

SEED = 123

## 데이터 로드


In [ ]:
test = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
test.head(10)

In [ ]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

# 이미지 전처리 정의
transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),  # 이미지 크기 조정
        transforms.ToTensor(),  # 이미지를 텐서로 변환
    ]
)

# ImageFolder를 사용하여 데이터셋 로드
train_dataset = datasets.ImageFolder(root="data/train", transform=transform)

# train / validation 비율 설정 (8:2)
train_size = int(0.8 * len(train_dataset))
validation_size = len(train_dataset) - train_size

# 데이터셋 분할
train_dataset, validation_dataset = random_split(
    train_dataset, [train_size, validation_size]
)

# DataLoader 인스턴스 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

In [ ]:
x, y = next(iter(train_loader))
print(x.shape, y.shape)

In [ ]:
x, y = next(iter(validation_loader))
print(x.shape, y.shape)

In [ ]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader


class TestDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): csv 파일의 경로.
            root_dir (string): 모든 이미지 파일이 있는 디렉토리의 경로.
            transform (callable, optional): 샘플에 적용될 Optional transform.
        """
        self.test_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.test_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.test_frame.iloc[idx, 0])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image


# 이미지 전처리 정의
transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),  # 이미지 크기 조정
        transforms.ToTensor(),  # 이미지를 텐서로 변환
    ]
)

# TestDataset 인스턴스 생성
test_dataset = TestDataset(
    csv_file="data/test.csv", root_dir="data/test", transform=transform
)

# DataLoader 인스턴스 생성
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
x = next(iter(test_loader))
print(x.shape)

## Label 확인


In [ ]:
classes = train_dataset.dataset.classes
print(classes)

idx_to_class = {v: k for k, v in train_dataset.dataset.class_to_idx.items()}
print(idx_to_class)

## 시각화


In [ ]:
import torchvision

# 한 배치의 이미지 시각화 함수 (사이즈 조정 포함)


def imshow(img, labels, classes):
    img = img.numpy().transpose((1, 2, 0))
    plt.figure(figsize=(20, 20))  # 여기에서 플롯의 크기를 조정
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    # 이미지마다 클래스 레이블을 타이틀로 표시
    for i, label in enumerate(labels):
        x = (i % 8) * (img.shape[1] / 8) + (img.shape[1] / 16)
        y = (i // 8) * (img.shape[0] / 4) + 10  # 4 rows
        plt.text(
            x,
            y,
            classes[label],
            ha="center",
            va="top",
            color="white",
            fontsize=12,
            backgroundcolor="black",
        )
    plt.show()


# 데이터 로더에서 한 배치 가져오기
dataiter = iter(train_loader)
images, labels = next(dataiter)

# 이미지 그리드 만들기
img_grid = torchvision.utils.make_grid(images, nrow=8)  # 8개의 이미지를 한 줄에 표시

# 이미지와 레이블 시각화
imshow(img_grid, labels, classes)

## 모델 정의


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import time

# CUDA 사용 가능 여부 확인
if torch.backends.mps.is_built():
    # mac os mps 지원 체크
    device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")
else:
    # cuda 사용 가능한지 체크
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        # 코드입력

    def forward(self, x):
        # 코드입력

        return x

In [ ]:
model = # 코드입력
model

In [ ]:
# 옵티마이저를 정의합니다. 옵티마이저에는 model.parameters()를 지정해야 합니다.
optimizer = # 코드입력

# 손실함수(loss function)을 지정합니다. Multi-Class Classification 이기 때문에 CrossEntropy 손실을 지정하였습니다.
loss_fn = # 코드입력

## 모델 훈련 및 검증


In [ ]:
# 코드입력

## Epoch 별 훈련


In [ ]:
import time

# 최대 Epoch을 지정합니다.
num_epochs = 10

min_loss = np.inf

STATE_DICT_PATH = "CATSvsDOGS-CNN.pth"

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    start = time.time()
    train_loss, train_acc = # 코드입력

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = # 코드입력

    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(
            f"[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!"
        )
        min_loss = val_loss
        torch.save(model.state_dict(), STATE_DICT_PATH)

    time_elapsed = time.time() - start
    # Epoch 별 결과를 출력합니다.
    print(
        f"[Epoch{epoch+1:02d}] time: {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s \t loss: {train_loss:.5f}, acc: {train_acc:.5f} | val_loss: {val_loss:.5f}, val_acc: {val_acc:.5f}"
    )

## 저장한 가중치 로드


In [ ]:
# 모델에 저장한 가중치를 로드합니다.
model.# 코드입력

## 예측 코드


In [ ]:
predictions = []
model = model.to(device)
# 검증모드 진입
model.eval()

with torch.no_grad():
    # loss 초기화
    running_loss = 0
    # 정확도 계산
    running_acc = 0
    for x in test_loader:
        x = x.to(device)

        y_hat = model(x)
        label = y_hat.argmax(dim=1).detach().item()
        predictions.append(label)

In [ ]:
# 정답
your_answer = [idx_to_class[l] for l in predictions]

## 결과 제출

- 느리다고 중지 후 다시 평가 코드를 실행하는 경우 제출 과정에서 패널티가 발생할 수 있습니다. (제출 횟수 이슈 발생 가능)
- 제출결과는 [대회페이지](http://braincrew2.iptime.org:8001/competitions/ADVHOUSE/)의 `리더보드` 와 `제출` 탭에서 확인할 수 있습니다.


아래 Cell을 실행하여 예측 결과 업데이트


In [ ]:
import competition

# 예측 결과 업데이트
submission = pd.read_csv(os.path.join(DATA_DIR, "submission.csv"))
submission["label"] = your_answer

display(submission)
competition.submit(project, username, password, submission)